In [2]:
import getpass
from tapipy.tapis import Tapis

In [4]:
username = getpass.getpass(prompt = "Username: ", stream=None)
password = getpass.getpass(prompt = "Password: ", stream=None)

base_url = "https://tacc.tapis.io"

client = Tapis(
    base_url = base_url, 
    username = username,
    password = password
) 

#generate access token
client.get_tokens()

Username: ········
Password: ········


In [8]:
project = {
    "project_name": f"smart_data_workshop_{username}",
    "description": "A smart data workshop project for user %s" % username,
    "owner": username,
    "pi": username,
    "active": True,
    "metadata": {}
}
client.streams.create_project(**project)


active: True
description: A smart data workshop project for user mcleanj
metadata: 

owner: mcleanj
permissions: 
users: ['mcleanj']
pi: mcleanj
project_id: smart_data_workshop_mcleanj
project_name: smart_data_workshop_mcleanj

In [ ]:
site = {
    "project_id": "project[],
    "site_name": "smart_data_workshop_%s" % username,
    "site_id": "smart_data_workshop_%s" % username,
    "description": "A smart data workshop project for user %s" % username,
    "owner": username,
    "pi": username,
    "active": True,
    "metadata": {}
}
client.streams.create_site(**site)

In [ ]:
instrument = {
    "project_id": "project[],
    "site_name": "smart_data_workshop_%s" % username,
    "site_id": "smart_data_workshop_%s" % username,
    "description": "A smart data workshop project for user %s" % username,
    "owner": username,
    "pi": username,
    "active": True,
    "metadata": {}
}
client.streams.create_project(**instrument)

In [ ]:
variable = {
    "project_id": "project[],
    "site_name": "smart_data_workshop_%s" % username,
    "site_id": "smart_data_workshop_%s" % username,
    "description": "A smart data workshop project for user %s" % username,
    "owner": username,
    "pi": username,
    "active": True,
    "metadata": {}
}
client.streams.create_project(**variable)

In [ ]:
measurements = {
    
}


In [13]:
actor = {
    "image": "abacosamples/test",
    "name": "test",
    "description": "test"
}

actor_info = client.actors.createActor(**actor)
actor_id = actor_info.id
print(actor_id)

5OejkbjJJEj4r


In [14]:
exec_info = client.actors.sendMessage(actor_id = actor_id, request_body = {
    "message": "test message"
})
exec_id = exec_info.executionId

In [20]:
execution = client.actors.getExecution(actor_id = actor_id, execution_id = exec_id)
print(execution)


_links: 
logs: https://tacc.tapis.io/v3/actors/5OejkbjJJEj4r/executions/EV84ezrXJoAq8/logs
owner: https://tacc.tapis.io/v3/oauth2/profiles/mcleanj
actorId: 5OejkbjJJEj4r
apiServer: https://tacc.tapis.io
cpu: 0
executor: mcleanj
id: EV84ezrXJoAq8
io: 0
messageReceivedTime: 2021-12-17T02:41:22.266Z
runtime: 0
status: ERROR
workerId: wKVNrpNWDeljq


In [21]:
logs = client.actors.getExecutionLogs(actor_id = actor_id, execution_id = exec_id)
print(logs)


_links: 
execution: https://tacc.tapis.io/v3/actors/5OejkbjJJEj4r/executions/EV84ezrXJoAq8
owner: https://tacc.tapis.io/v3/oauth2/profiles/mcleanj
logs: 
